In [13]:
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like

from pandas_datareader import data as pdata
import matplotlib.pyplot as plt
import datetime, urllib3,json
import datetime,warnings

%matplotlib inline


import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)


def get_etf_constituents(etf_tickers):
    res={}
    http = urllib3.PoolManager()
    for etf in etf_tickers:
        stock_tickers=[]
        r = http.request('GET', 'https://core-api.barchart.com/v1/EtfConstituents?'+
        'composite=%s&'%etf+
        'fields=symbol,symbolName,percent,sharesHeld,symbolCodes,symbolType,lastPrice,dailyLastPrice&'+
        'orderBy=percent&orderDir=desc&meta=field.shortName,field.type,field.description&page=1&'+
        'limit=100&'+
        'raw=1')
        datas = json.loads(r.data.decode('utf-8'))['data']

        #print("datas ", datas)
        for data in datas:
            if (data['symbol'] != 'Unassigned') and \
                (data['symbol'] != '') and (data['symbol'] != etf):
                stock_tickers.append(data['symbol'])


        res[etf]=stock_tickers
    #print("append etc dict ", res)
    return res


#etfs=['XLF']

def check_all_etfs_corr(etfs, all_stocks_px, corr_thd =[0.9,0.9,0.9]):

    etf_dict =get_etf_constituents(etfs)

    #end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    #start_date = (end_dt-datetime.timedelta(days=(window*2))).strftime("%Y-%m-%d")
    etf_set={}
    for key, value in etf_dict.items():
        df_sector=[]
        print(key)
        df_sector=pd.DataFrame()
        for symbol in value:
            if symbol in all_stocks_px.columns.values:
                df_sector[symbol]= all_stocks_px[symbol]
        #print("df sector ",df_sector)
        #df = pdata.get_data_yahoo(value, start_date, end_date)

        etf_set[key]=get_high_corr_stocks(df_sector,key, corr_thd)
        #print("etf_ set", etf_set)
    return etf_set

def get_corr_matrix(df, window,price_type='Close'):
    for win in window:
        if df.shape[0]< win:
            warnings.warn(" Increase window size ")
            return

    price_df = df
    dates = price_df.index.values
    symbol =price_df.columns.values
    price =  price_df.as_matrix()
    corr_mtxs={}

    for win in window:
        corr_mtxs[win] = np.corrcoef(price[-win:], rowvar=False) #col is variable

    #print(corr_mtx)
    return dates, symbol, corr_mtxs


def get_high_corr_stocks(df,sector=None, corr_thd = [0.9,0.9,0.8], window=[120,60,20]):

    print ( "===== correlation thd %f====== "%corr_thd[0], corr_thd)
    corr_period={}

    dates, symbols, corr_mtxs = get_corr_matrix(df,window)

    res=set()
    for i in range (len(symbols)):
        for j in range (i+1,len(symbols)):
            #print ("index ", i, j)
            is_corr_stable = True
            for k in range(len(window)):
                #print ("index ", i,symbols[i], j,symbols[j],win, corr_mtxs[win][i,j] )

                #print(type(corr_mtxs[win][i,j]))
                if corr_mtxs[int(window[k])][i,j]<corr_thd[k]:
                    #print ("false  ", i,symbols[i], j,symbols[j], win, corr_mtxs[win][i,j])
                    is_corr_stable = False
                    break
                elif np.isnan(corr_mtxs[int(window[k])][i,j]):
                    is_corr_stable = False
                    break
                    #print ("false  ", i,symbols[i], j,symbols[j], win, corr_mtxs[win][i,j])
                #else:
                  #print ("index ", i,symbols[i], j,symbols[j],win, corr_mtxs[win][i,j] )

        if is_corr_stable:
            res.update([symbols[i],symbols[j]])
            #print ("index ", i,symbols[i], j,symbols[j] )



        #print(res)
        #corr_period[win]=res.copy()

    print("======HIGHLY CORR SET (%s)======"%sector)
    #print(res)
    return list(res)

'''
UTILS
'''
def getOpenCloseDf(stockprice_panel, include_open=False):
    
    result=pd.DataFrame()
    symbols= stockprice_panel.minor_axis
    length = stockprice_panel.shape[1]
    
    for symbol in symbols:
        
        if include_open :
            open_close_px=np.empty([2*length])
            ts=np.empty([2*length])
            open= stockprice_panel.minor_xs(symbol)['Open'].values
            close = stockprice_panel.minor_xs(symbol)['Close'].values
            for i in range (0,len(close)):
                open_close_px[i*2]=open[i]
                open_close_px[i*2+1]=close[i]
                ts[i*2]= int(stockprice_panel.minor_xs(symbol).index[i].timestamp()+9.5*60*60)
                ts[i*2+1]= int(stockprice_panel.minor_xs(symbol).index[i].timestamp()+16*60*60)
            result[symbol]=open_close_px
        else:
            open_close_px=np.empty([length])
            ts=np.empty([length])
            close = stockprice_panel.minor_xs(symbol)['Close'].values
            for i in range (0,len(close)):
                open_close_px[i]=close[i]
                ts[i]= int(stockprice_panel.minor_xs(symbol).index[i].timestamp()+16*60*60)
            result[symbol]=open_close_px
            
        
    result.index=ts

    #print(result)
    return result

'''
UTILS
'''
def plot_price_series(df, ts1, ts2):
    #months = mdates.MonthLocator()  # every month
    fig, ax = plt.subplots()
    ax.plot(df.index, ts1, label=ts1.name,color='b')
    ax.set_ylabel(ts1.name, color='b')
    ax.tick_params('y', colors='b')
    #ax.plot(df.index, ts2, label=ts2.name)
    ax.grid(True)
    
    ax2 = ax.twinx()
    ax2.plot(df.index, ts2, label=ts2.name,color='r')
    ax2.set_ylabel(ts2.name, color='r')
    ax2.tick_params('y', colors='r')
    
    fig.autofmt_xdate()

    plt.xlabel('Day')
    #plt.ylabel('Price ($)')
    plt.title('%s and %s Daily Prices' % (ts1.name, ts2.name))
    #plt.legend()
    plt.show()


import statsmodels.tsa.stattools as st
import statsmodels.api as sm
#result = ts.coint(ts1, ts2)

#print ("coin test - sig lvl 5%", result)


def coint_finder(all_stck_df, coint_thd=0.01):
    stck_size = len(all_stck_df.columns)
    print("stock size ",stck_size, len(all_stck_df))
    pairs=[[]]
    signal_count =0
    for i in range (0,stck_size):
        for j in range (i+1, stck_size):
            ts1 = all_stck_df[ list(all_stck_df)[i]]
            ts2 = all_stck_df[ list(all_stck_df)[j]]

            coin_test = st.coint(ts1, ts2)
            #print(coin_test)
            if ( coin_test[1]< coint_thd ): #pvalue,  If the pvalue is small, below a critical size, then we can reject the hypothesis that there is no cointegrating relationship.

                print("coin_test %s and %s "%(ts1.name, ts2.name),coin_test)

                x=ts2
                y=ts1
                x=sm.add_constant(x)
                res = sm.OLS(y,x).fit()
                print ("correlation ")
                for k in [120, 60 ,20]:
                    if len(ts1)<k:
                        continue
                    corr = np.corrcoef(ts1.tail(k), ts2.tail(k))
                    print ("(%i):"%k, corr[0][1])

                print("OLS: ", res.params)

                base_url= "https://www.tradingview.com/chart/?symbol="
                print("check chart: https://www.tradingview.com/chart/?symbol=%s-(1*%s)*%s"%(ts1.name,res.params[1],ts2.name))

                fig, ax = plt.subplots()
                ax.plot(ts1.index,ts1-res.params[1]*ts2,label=ts1.name+"-"+ts2.name)
                ax.grid(True)
                fig.autofmt_xdate()
                plt.legend()
                plt.show()

                plot_price_series(all_stck_df,ts1,ts2)

                pairs.append([ts1.name, ts2.name])
                print("===============")

                signal_count+=1
            else:
                pass
    print("Signal count ", signal_count)

In [10]:
#rolling standerdize error

def get_rolling_stdize_error():
    pass

In [6]:
#MAIN 

def __main__(etfs, hist_win, check_win,
             end_date=datetime.date.today().strftime("%Y-%m-%d"),
             include_open=False,  corr_thd = [0.9,0.9,0.9],coint_thd=0.01):

    #end_date=datetime.date.today().strftime("%Y-%m-%d")
    end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    start_date = (end_dt-datetime.timedelta(days=(hist_win*2))).strftime("%Y-%m-%d")

    stockprice=None

    symbol=[]
    for etf in etfs:
        symbol.extend(get_etf_constituents([etf])[etf])

    stockprice = pdata.get_data_yahoo(symbol, start_date, end_date)
    print(stockprice.shape)
    
    normalized_px = stockprice['Close']
    
    if include_open:
        check_win *=2
        normalized_px=getOpenCloseDf(stockprice,True)
    #else 
        #normalized_px = getOpenCloseDf(stockprice,False)

    high_corr = check_all_etfs_corr(etfs,normalized_px, corr_thd)

    df_sector=pd.DataFrame()
    for key, value in high_corr.items():
        df_sector = pd.DataFrame(normalized_px, columns=value)
        for symbol in value:
            if symbol in normalized_px.columns.values:
                df_sector[symbol]= normalized_px[symbol]
        print("-----SECTOR %s-----"%key)
        df_sector= df_sector.tail(-check_win)
        coint_finder(df_sector,coint_thd) # use raw price to do cointergation and regression
        
        
#RUN
etfs=['XLF','XLE','XLI','XLB','XLP','XLY','XTL','XLK','XLV','XLU','XLRE']

#__main__(['XLF'],hist_win =300,check_win=120,
         
#         include_open=True,
#         corr_thd = [0.5,0.5,0.5],coint_thd=0.01 )

In [14]:
window=300
etfs=['XLF','XLE','XLI','XLB','XLP','XLY','XTL','XLK','XLV','XLU','XLRE']
etfs=['XLF']

end_date=datetime.date.today().strftime("%Y-%m-%d")
end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
start_date = (end_dt-datetime.timedelta(days=(window*2))).strftime("%Y-%m-%d")


stockprice=None

symbol=[]
#FIXME get_etf_constituents
#for etf in etfs:
#    symbol.extend(get_etf_constituents([etf])[etf])

symbol =['BAC','WFC','C','USB','GS','AXP',
'BRK.B','JPM','CME','MS',
'PNC','CB','SCHW','BLK',
'SPGI','BK',
'ICE','COF',
'MMC','MET','PGR',
'PRU','AIG',
'BBT','AON','TRV',
'AFL','ALL',
'STI','STT',
'MCO','DFS',
'TROW',
'MTB','NTRS',
'SYF','KEY',
'RF','WLTW',
'AMP',
'FITB','CFG',
'HIG','HBAN',
'CMA','AJG',
'LNC',
'L','ETFC',
'MSCI',
'SIVB','PFG',
'CBOE','CINF',
'RJF','NDAQ',
'BEN','ZION',
'IVZ',
'RE','TMK',
'UNM','AMG',
'JEF','PBCT',
'AIZ','BHF',]

print(" symbol ", symbol)
stockprice = pdata.get_data_yahoo(symbol, start_date, end_date)
print(stockprice.shape)
#stockprice=stockprice['Close']

 symbol  ['BAC', 'WFC', 'C', 'USB', 'GS', 'AXP', 'BRK.B', 'JPM', 'CME', 'MS', 'PNC', 'CB', 'SCHW', 'BLK', 'SPGI', 'BK', 'ICE', 'COF', 'MMC', 'MET', 'PGR', 'PRU', 'AIG', 'BBT', 'AON', 'TRV', 'AFL', 'ALL', 'STI', 'STT', 'MCO', 'DFS', 'TROW', 'MTB', 'NTRS', 'SYF', 'KEY', 'RF', 'WLTW', 'AMP', 'FITB', 'CFG', 'HIG', 'HBAN', 'CMA', 'AJG', 'LNC', 'L', 'ETFC', 'MSCI', 'SIVB', 'PFG', 'CBOE', 'CINF', 'RJF', 'NDAQ', 'BEN', 'ZION', 'IVZ', 'RE', 'TMK', 'UNM', 'AMG', 'JEF', 'PBCT', 'AIZ', 'BHF']


ConnectionError: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Max retries exceeded with url: /quote/SPY/history (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x122879978>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
# MAIN - generate plot 

#SET-UP
include_open = False
window =300
corr_thd = [0.0,0.0,0.0]
coint_thd=0.05

#RUN
normalized_px = getOpenCloseDf(stockprice)
if include_open:
    window *=2
    normalized_px=getOpenCloseDf(stockprice,True)

high_corr = check_all_etfs_corr(etfs,normalized_px, corr_thd)

df_sector=pd.DataFrame()
for key, value in high_corr.items():
    df_sector = pd.DataFrame(normalized_px, columns=value)
    for symbol in value:
        if symbol in normalized_px.columns.values:
            df_sector[symbol]= normalized_px[symbol]
    print("-----SECTOR %s-----"%key)
    df_sector= df_sector.tail(window)
    coint_finder(df_sector,coint_thd) # use raw price to do cointergation and regression